In [ ]:
import os, logging
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Concatenate, Activation, BatchNormalization, Dropout, Conv1D
from keras import optimizers, callbacks, initializers, constraints, regularizers
from keras.models import Model
from libs.hydrolayer import PhyRNNLayer, Conv1Layer, Conv2Layer, ScaleLayer
from libs.hydrodata import DataforIndividual
from libs import hydroutils
from datetime import datetime
from matplotlib import pyplot as plt

## Ignore all the warnings
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['KMP_WARNINGS'] = '0'

In [ ]:
working_path = '/home/shijie/phydeep'
basin_id = '01013500'
####
training_start = '1980-10-01'
training_end= '2000-09-30'
####
val_start = '1999-10-01'
val_end= '2010-09-30'
####
testing_start = '2000-10-01'
testing_end= '2010-09-30'

In [ ]:
def generate_train_test(train_set, val_set, test_set, wrap_length=1825):
    train_x_np = train_set.values[:, :-1]
    train_y_np = train_set.values[:, -1:]
    test_x_np = test_set.values[:, :-1]
    test_y_np = test_set.values[:, -1:]
    val_x_np = val_set.values[:, :-1]
    val_y_np = val_set.values[:, -1:]
    
    
    wrap_number_train = (train_set.shape[0]-wrap_length)//365 + 1
    wrap_number_val = (val_set.shape[0]-wrap_length)//365 + 1
    
    train_x = np.empty(shape = (wrap_number_train, wrap_length, train_x_np.shape[1]))
    train_y = np.empty(shape = (wrap_number_train, wrap_length, train_y_np.shape[1]))
    val_x = np.empty(shape = (wrap_number_val, wrap_length, val_x_np.shape[1]))
    val_y = np.empty(shape = (wrap_number_val, wrap_length, val_y_np.shape[1])) 
    test_x = np.expand_dims(test_x_np, axis=0)
    test_y = np.expand_dims(test_y_np, axis=0)
    
    for i in range(wrap_number_train):
        train_x[i, :, :] = train_x_np[i*365:(wrap_length+i*365), :]
        train_y[i, :, :] = train_y_np[i*365:(wrap_length+i*365), :]

    for i in range(wrap_number_val):
        val_x[i, :, :] = val_x_np[i*365:(wrap_length+i*365), :]
        val_y[i, :, :] = val_y_np[i*365:(wrap_length+i*365), :]        
        
    return train_x, train_y,val_x,val_y,  test_x, test_y

def create_model(input_shape, seed, hidden_nodes):
    x_input = Input(shape=input_shape, name='Input')
    hydro = PhyRNNLayer(mode= 'normal', name='Hydro')(x_input)
    
    x_hydro = Concatenate(axis=-1, name='Concat')([x_input, hydro])
    x_scale = ScaleLayer(with_centering=False, with_scaling=False, name='Scale')(x_hydro)
    
    output = Conv1Layer(filters=hidden_nodes, kernel_size=1, padding='causal', seed=seed, name='Conv1')(x_scale)
    output = Conv2Layer(filters=1, kernel_size=10, padding='causal', seed=seed, name='Conv2')(output)

    model = Model(x_input, hydro)
    return model

def train_model(model, train_x, train_y, val_x, val_y, ep_number, lrate, save_path):
    save = callbacks.ModelCheckpoint(save_path, verbose=0, save_best_only=True, monitor='nse_metrics', mode='max',
                                     save_weights_only=True)
    es = callbacks.EarlyStopping(monitor='nse_metrics', mode='max', verbose=1, patience=20, min_delta=0.0005,
                                 restore_best_weights=True)
    reduce = callbacks.ReduceLROnPlateau(monitor='nse_metrics', factor=0.8, patience=5, verbose=1, mode='max',
                                         min_delta=0.0005, cooldown=0, min_lr=lrate / 100)
    tna = callbacks.TerminateOnNaN()

    model.compile(loss=hydroutils.nse_loss, metrics=[hydroutils.nse_metrics], optimizer=optimizers.Adam(lr=lrate))
    history = model.fit(train_x, train_y, epochs=ep_number, batch_size=10000, callbacks=[save, es, reduce, tna],
                       validation_data=[val_x, val_y])
    
    return history
    
def test_model(model, test_x, test_y, save_path):
    model.load_weights(save_path, by_name=True)
    pred_y = model.predict(test_x, batch_size=10000)
    
    return pred_y

def calc_nse(obs: np.array, sim: np.array) -> float:
    denominator = np.sum((obs - np.mean(obs)) ** 2)
    numerator = np.sum((sim - obs) ** 2)
    nse_val = 1 - numerator / denominator

    return nse_val

In [ ]:
hydrodata = DataforIndividual(working_path, basin_id).load_data()

train_set = hydrodata[hydrodata.index.isin(pd.date_range(training_start, training_end))]
val_set = hydrodata[hydrodata.index.isin(pd.date_range(val_start, val_end))]
test_set = hydrodata

train_x, train_y,val_x,val_y,  test_x, test_y = generate_train_test(train_set,val_set, test_set, wrap_length=1825)

In [ ]:
#####
train_x_mean = train_set.mean()[:-1].values
train_y_mean = train_set.mean()[-1:].values
train_x_std = train_set.std()[:-1].values
train_y_std = train_set.std()[-1:].values
#####
train_x_np = (train_set.values[:, :-1] - train_x_mean) / train_x_std
train_y_np = (train_set.values[:, -1:] - train_y_mean) / train_y_std
test_x_np = (test_set.values[:, :-1] - train_x_mean) / train_x_std
test_y_np = (test_set.values[:, -1:] - train_y_mean) / train_y_std

plt.plot(test_y_np[:,0])

In [ ]:
model = create_model((train_x.shape[1], train_x.shape[2]), 200, 32)
model.summary()

In [ ]:
save_path = f'{working_path}/{basin_id}.h5'
history = train_model(model, train_x, train_y, val_x, val_y, 500, 0.01, save_path)

In [ ]:
save_path = '{0}/{1}/{2}.h5'.format(working_path, 'single_results/white_model', basin_id)

x_input = Input(shape=(test_x.shape[1], test_x.shape[2]), name='Input')
hydro = PhyRNNLayer(mode= 'normal', name='Hydro')(x_input)

x_hydro = Concatenate(axis=-1, name='Concat')([x_input, hydro])
x_scale = ScaleLayer(with_centering=False, with_scaling=False, name='Scale')(x_hydro)

output = Conv1Layer(filters=16, kernel_size=1, padding='causal', seed=200, name='Conv1')(x_scale)
output = Conv2Layer(filters=1, kernel_size=10, padding='causal', seed=200, name='Conv2')(output)

model_for_test = Model(x_input, hydro)

pred_y = test_model(model_for_test, test_x, test_y, save_path)


In [ ]:
evaluate_set = test_set.loc[:, ['prcp(mm/day)','flow(mm)']]
evaluate_set['flow_obs'] = evaluate_set['flow(mm)']
evaluate_set['flow_sim'] = np.clip(pred_y[0, :, :], a_min = 0, a_max = None)

####
training_start = '1980-10-01'
training_end= '2000-09-30'
####
testing_start = '2000-10-01'
testing_end= '2010-09-30'

date_range = pd.date_range(testing_start, testing_end)
evaluate_set = evaluate_set[evaluate_set.index.isin(date_range)]
nse = calc_nse(evaluate_set['flow_obs'].values, evaluate_set['flow_sim'].values)

plot_set = evaluate_set[evaluate_set.index.isin(pd.date_range(training_start, testing_end))]
fig, ax = plt.subplots(figsize=(15, 4.5))

ax.plot(plot_set['flow_sim'], label="simulation")
ax.plot(plot_set['flow_obs'], label="observation")


ax.set_title(f"Basin {basin_id} - Test set NSE: {nse:.3f}")
ax.set_ylabel("Streamflow (mm/day)")
ax.legend()
plt.show()

In [ ]:
plt.plot(pred_y[0, :, 3], label="observation")

In [ ]:
model.get_weights()[0:12]

In [ ]:
(0.71950805*1900+100)